### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [10]:
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings

#Para gerar uma chave API Cohere, basta entrar em https://dashboard.cohere.com/api-keys; 
#Há tanto uma API Trial (gratuita e mais limitada) quanto uma API Produtiva

cohere = CohereEmbeddings(cohere_api_key=cohere_api_key, model = "multilingual-22-12")

#Nome da pasta de onde está a VectorStore
persist_directory = 'InnovationCamp_VectorDB'

#Carregando a VectorStore do disco
vectorstore = Chroma(persist_directory=persist_directory,
                  embedding_function=cohere)


In [2]:
question = "Como previnir câncer de mama?"
docs = vectorstore.similarity_search(question)

In [3]:
docs[0].page_content

'que favorecem o desenvolvimento desse tipo de câncer caem na mulher. Além disso, na amamentação também ocorre renovação de células que poderiam ter lesões, diminuindo assim as chances da doença. Quanto mais prolongada for a amamentação, maior a proteção para a mãe. Dica bônus: realizar mamografia. Além de prevenção, é fundamental falarmos de detecção precoce. É imprescindível que as mulheres a partir dos 40 anos realizem anualmente a mamografia, exame mais eficaz para o diagnóstico precoce. Nas'

In [4]:
docs[0].metadata['source']

'https://www.pucrs.br/blog/prevencao-cancer-de-mama/'

In [11]:
#Apenas para formatar as fontes. Como o Retriever retorna os top K resultados, podemos ter K fontes iguais e
#não queremos printar a mesma fonte K vezes. 
def unique_elements(input_list):
    unique_elements = list(set(input_list))
    return unique_elements

In [12]:
def format_answer(fontes, answer):
    unique_fontes = unique_elements(fontes)
    
    fontes_formatadas = ''
    
    if len(unique_fontes) == 1:
        for i in range(len(unique_fontes)):
            fontes_formatadas += f"{i+1}. {unique_fontes[i]}\n "
    
            resultado_final = f"\nResposta do Modelo: {answer} \n \n Leia mais no link: \n {fontes_formatadas}"
    
    elif len(unique_fontes) > 1:
        for i in range(len(unique_fontes)):
            fontes_formatadas += f"{i+1}. {unique_fontes[i]}\n "

        resultado_final = f"\nRosana: {answer} \n \n Leia mais nos links: \n {fontes_formatadas}"
        
    return resultado_final

In [13]:
import maritalk

#Para gerar uma chave API, basta entrar em https://chat.maritaca.ai/, criar uma conta e criar a API.

model = maritalk.MariTalk(key=maritalk_key)

In [14]:
def print_info(hospital):
    print(f"Nome do Hospital: {hospital.Hospital}")
    print(f"Endereço: {hospital.Endereço}, CEP {hospital.CEP}")
    print(f"Telefone: {hospital.Telefone}")
    print(f"Site: {hospital.Site}")

In [15]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
from colorama import Fore

In [16]:
hospital_excel = pd.read_excel("2023.12.07 - IA e Tech - Lista de hospitais.xlsx")

In [17]:
def print_disclaimer():
    
    disclaimer = '''    *Disclaimer*:
    
    Como toda Inteligência Artificial, eu também posso cometer erros e retornar informações imprecisas ou até mesmo incorretas. 
    É fortemente recomendada a leitura as fontes fornecidas pois nelas temos informações curadas, seguras e corretas, além de 
    muitas outras informações que podem não estar contidas na minha resposta. 
    
    Lembre-se: A melhor forma de prevenção contra o câncer de mama é a informação! 
    '''
    print(disclaimer)


In [18]:
def sugere_hospital_proximo(hospital_excel):
    geocoder = Nominatim(user_agent="user_agent")
    endereço = str(input("Digite seu endereço: "))
    print("")
    location = geocoder.geocode(endereço)

    lat = location.latitude
    long = location.longitude

    dist_0 = 2000 #número alto genérico
    i_min = 0

    for i in range(len(hospital_excel)):

        hospital_lat = hospital_excel.iloc[i].Latitude
        hospital_long = hospital_excel.iloc[i].Longitude


        dist = geodesic((lat,long), (hospital_lat, hospital_long)).kilometers

        if dist<dist_0:
            dist_0 = dist
            i_min = i

    hospital_mais_proximo = hospital_excel.iloc[i_min]

    print("A unidade mais próxima do endereço fornecido e que atende seu convênio é: ")
    print("")
    print_info(hospital_mais_proximo)

In [19]:
pergunta = str(input("Digite sua pergunta: "))

docs_2 = vectorstore.similarity_search(pergunta, k=4)

contexto_2 = ''
fontes_2 = []

for i in range (len(docs_2)):
    contexto_2 += f"{docs_2[i].page_content} "
    fontes_2.append(docs_2[i].metadata['source'])
    
prompt = f'''A pergunta abaixo tem relação com informações de câncer de mama. 
Responda, por favor, de forma direta, concisa e resumida a pergunta a seguir baseado somente no contexto fornecido.
Se não for possível deduzir a resposta adequada a partir do contexto, responda "Eu não sei" e não invente uma resposta. 
Evite a todo custo fornecer diagnósticos sobre a saúde do paciente, e reforce a importância de consultar ajuda médica.
Responda com atenção pois isto é muito importante para minha carreira.
Contexto: {contexto_2};
Pergunta: {pergunta};
'''

answer = model.generate(
    prompt,
    do_sample=True,
    max_tokens=1000,
    temperature=0.1,
    top_p=0.95)

resultado_final = format_answer(fontes_2, answer)

print(resultado_final)

quer_sugestão = str(input("Gostaria de encontrar o hospital mais próximo de um endereço? [y/n]:"))
if quer_sugestão == 'y':
    intention = str(input("Possui algum convênio? [y/n]:"))
    
    if intention == 'y':
        convenio = str(input("Digite o nome do seu convênio: "))
        
        for i in range(len(hospital_excel)):
            if convenio not in hospital_excel.Convênios.iloc[i]:
                hospital_excel.drop([i])  

    sugere_hospital_proximo(hospital_excel)
else:
    pass
print("")
print_disclaimer()

Digite sua pergunta:  Mamografia dói?



Rosana: Sim, a mamografia pode causar algum desconforto ou dor durante o exame. Isso ocorre porque o aparelho de mamografia comprime a mama para obter uma imagem nítida. No entanto, a dor é geralmente tolerável e temporária. Se você estiver preocupado com a dor, converse com seu médico ou técnico de mamografia antes do exame. Eles podem explicar o procedimento e ajudá-lo a se sentir mais confortável. 
 
 Leia mais nos links: 
 1. https://www.pucrs.br/blog/prevencao-cancer-de-mama/
 2. https://www.cancerdemamabrasil.com.br/como-funciona-a-mamografia/#:~:text=A%20mamografia%20utiliza%20um%20aparelho,mama%20para%20separar%20o%20tecido.
 3. http://www.oncoguia.org.br/conteudo/sinais-e-sintomas-do-cancer-de-mama/1383/34/
 


Gostaria de encontrar o hospital mais próximo de um endereço? [y/n]: y
Possui algum convênio? [y/n]: Av. Portugal, 1290
Digite seu endereço:  n



A unidade mais próxima do endereço fornecido e que atende seu convênio é: 

Nome do Hospital: Hospital Albert Einstein - Morumbi
Endereço: Av. Albert Einstein, 627/701 - Morumbi, São Paulo, CEP 05652-900
Telefone: (11) 2151-1233
Site: https://www.einstein.br/

    *Disclaimer*:
    
    Como toda Inteligência Artificial, eu também posso cometer erros e retornar informações imprecisas ou até mesmo incorretas. 
    É fortemente recomendada a leitura as fontes fornecidas pois nelas temos informações curadas, seguras e corretas, além de 
    muitas outras informações que podem não estar contidas na minha resposta. 
    
    Lembre-se: A melhor forma de prevenção contra o câncer de mama é a informação! 
    
